In [3]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from costs import *
from models import *
from helpers import *
from build import *

## Load the training data into feature matrix, class labels, and event ids:

In [5]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)

In [6]:
print(y.shape)
head = range(10)
print(y[head])
print(tX[ range(2) ])
print(ids[head])

(5000,)
[ 1. -1. -1.  1.  1.  1. -1. -1.  1. -1.]
[[  1.38470000e+02   5.16550000e+01   9.78270000e+01   2.79800000e+01
    9.10000000e-01   1.24711000e+02   2.66600000e+00   3.06400000e+00
    4.19280000e+01   1.97760000e+02   1.58200000e+00   1.39600000e+00
    2.00000000e-01   3.26380000e+01   1.01700000e+00   3.81000000e-01
    5.16260000e+01   2.27300000e+00  -2.41400000e+00   1.68240000e+01
   -2.77000000e-01   2.58733000e+02   2.00000000e+00   6.74350000e+01
    2.15000000e+00   4.44000000e-01   4.60620000e+01   1.24000000e+00
   -2.47500000e+00   1.13497000e+02]
 [  2.19057000e+02   7.24610000e+01   1.24835000e+02   5.50600000e+00
   -9.99000000e+02  -9.99000000e+02  -9.99000000e+02   3.77100000e+00
    4.69360000e+01   1.22986000e+02   1.93200000e+00  -1.38200000e+00
   -9.99000000e+02   2.47590000e+01  -1.06300000e+00   3.32000000e-01
    4.78300000e+01   1.34700000e+00  -2.56900000e+00   2.84990000e+01
    9.60000000e-01   9.03550000e+01   1.00000000e+00   5.03960000e+01
   

## Do your thing crazy machine learning thing here :) ...

Let's begin with a simple linear regression with least_square gradient descent

In [7]:
gamma = 1.0
max_iter = 10
loss, w = least_squares_SGD(y, tX, gamma, max_iter)
w

array([ -2.56608363e+59,   1.10782626e+59,   1.62451845e+59,
         6.26336910e+58,  -2.01610081e+60,  -2.04793050e+60,
        -2.01579484e+60,   5.04332859e+57,   2.88293272e+58,
         1.96937732e+59,   2.82441729e+57,  -1.06706918e+57,
        -2.01597045e+60,   6.86296372e+58,   1.18364804e+55,
         7.30831677e+54,   8.65034239e+58,  -5.15820142e+55,
         2.15281385e+56,   7.01699928e+58,  -2.88226035e+54,
         2.97018336e+59,   6.70599200e+56,  -1.20485188e+60,
        -1.25039788e+60,  -1.25035089e+60,  -2.01883416e+60,
        -2.01594418e+60,  -2.01593459e+60,   4.18046866e+58])

In [10]:
loss, w = reg_logistic_regression(y, tX)

Current iteration=0, the loss=[[ 3465.7359028]]
Calculation failed! [ 3003.90589018]


OverflowError: math range error

In [ ]:
gamma = 1.0
max_iter = 10
loss, w = least_squares_SGD(y, tX, gamma, max_iter)

In [ ]:
print(tX[ range(3)])
x_first_column = tX[:, 0]

In [ ]:
# let's do a prediction only about first column, just for fun
tx_first_column = build_poly(x_first_column, 2)
loss, w = least_squares_SGD(y, tX, gamma, 15)

In [ ]:
print(len(w))
print(loss)

In [ ]:
print(tX.shape)
np.var(tX, axis=0)

Let's try some cross-validation, same as the HW4

In [ ]:
from costs import *
from build_polynomial import build_poly

def cross_validation(y, tX, k_indices, k, lambda_):
    """return the loss of ridge regression."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # get k'th subgroup in test, others in train: TODO
    # ***************************************************
    ytest = y[k_indices[k]]
    tXtest = tX[k_indices[k]]
    
    not_k = np.array([i for i in range(len(y)) if i not in k_indices[k]])
    ytrain = y[not_k]
    tXtrain = tX[not_k]

    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression: TODO
    # ***************************************************
    _, w = ridge_regression(ytrain, tXtrain, lambda_)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate the loss for train and test data: TODO
    # ***************************************************
    loss_tr = compute_loss(ytrain, tXtrain, w)
    loss_te = compute_loss(ytest, tXtest, w)
    

    return loss_tr, loss_te

In [ ]:
from plots import cross_validation_visualization
from build_k_indices import *

def cross_validation_demo():
    seed = 1
    k_fold = 4
    lambdas = np.logspace(-10, 2, 30)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    mse_tr = []
    mse_te = []
    # ***************************************************
    # INSERT YOUR CODE HERE
    # cross validation: TODO
    # *************************************************** 
    for lambda_ in lambdas:
        trainerrors = []
        testerrors = []
        for k in range(k_fold):
            trainerror, testerror = cross_validation(y, tX, k_indices, k, lambda_)
            trainerrors.append(trainerror)
            testerrors.append(testerror)
        
        mse_tr.append(np.mean(trainerrors))
        mse_te.append(np.mean(testerrors))
    
    
    cross_validation_visualization(lambdas, mse_tr, mse_te)

cross_validation_demo()

Then we choose a lambda near zéro (then just a least_square_GD), we will test our stupid regression with least_square_GD and variation of gamma

In [ ]:
from costs import *
from build_polynomial import build_poly
from plots import cross_validation_visualization
from build_k_indices import *

def cross_validation(y, tX, k_indices, k, gamma):
    """return the loss of ridge regression."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # get k'th subgroup in test, others in train: TODO
    # ***************************************************
    ytest = y[k_indices[k]]
    tXtest = tX[k_indices[k]]
    
    not_k = np.array([i for i in range(len(y)) if i not in k_indices[k]])
    ytrain = y[not_k]
    tXtrain = tX[not_k]

    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression: TODO
    # ***************************************************
    max_iters = 10
    loss, w = least_squares_GD(ytrain, tXtrain, gamma, max_iters)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate the loss for train and test data: TODO
    # ***************************************************
    loss_tr = compute_loss(ytrain, tXtrain, w)
    loss_te = compute_loss(ytest, tXtest, w)

    return loss_tr, loss_te


def cross_validation_demo():
    seed = 1
    k_fold = 4
    # lambdas = np.logspace(-10, 2, 30)
    gammas = np.arange(0.0, 1.0, 0.1)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    mse_tr = []
    mse_te = []
    # ***************************************************
    # INSERT YOUR CODE HERE
    # cross validation: TODO
    # *************************************************** 
    for gamma in gammas:
        trainerrors = []
        testerrors = []
        for k in range(k_fold):
            trainerror, testerror = cross_validation(y, tX, k_indices, k, gamma)
            trainerrors.append(trainerror)
            testerrors.append(testerror)
        
        mse_tr.append(np.mean(trainerrors))
        mse_te.append(np.mean(testerrors))
    
    cross_validation_visualization(gammas, mse_tr, mse_te)

cross_validation_demo()

Then we decide that a small gamma is ok, 0.5 seems good. We will just do one dummy prediction

In [ ]:
gamma = 0.5
max_iters = 10
loss, w = least_squares_GD(y, tX, gamma, max_iters)

In [ ]:
w

In [ ]:
w

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
weights = w

In [ ]:
OUTPUT_PATH = 'pred_test1.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)